In [1]:
import pandas as pd
import re
from sqlalchemy import create_engine
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import GridSearchCV
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Input, Bidirectional, SpatialDropout1D
from keras.layers.embeddings import Embedding

from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model

from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.initializers import RandomNormal

from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import log_loss
from keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
! unzip /content/drive/MyDrive/All/Participants_Data_DCW.zip

Archive:  /content/drive/MyDrive/All/Participants_Data_DCW.zip
  inflating: test.csv                
  inflating: __MACOSX/._test.csv     
  inflating: train.csv               
  inflating: __MACOSX/._train.csv    
  inflating: submission.csv          
  inflating: __MACOSX/._submission.csv  


In [3]:
df = pd.read_csv('/content/train.csv')

In [4]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

In [5]:
def clean_text(text):
    text = text.lower()
    
    
    text = re.sub(r'@[A-Za-z0-9_]+','', text)
    
    # Dealing with URL links
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    text = re.sub(url_regex,'urlplaceholder', text)
    # A lot of url are write as follows: http bit.ly. Apply Regex for these cases
    utl_regex_2 = 'http [a-zA-Z]+\.[a-zA-Z]+'
    text = re.sub(utl_regex_2,'urlplaceholder', text)
    # Other formats: http : //t.co/ihW64e8Z
    utl_regex_3 = 'http \: //[a-zA-Z]\.(co|com|pt|ly)/[A-Za-z0-9_]+'
    text = re.sub(utl_regex_3,'urlplaceholder', text)
    
   
    
    
    # Contractions
    text = re.sub(r"what's", 'what is ', text)
    text = re.sub(r"can't", 'cannot', text)
    text = re.sub(r"\'s",' ', text)
    text = re.sub(r"\'ve", ' have ', text)
    text = re.sub(r"n't", ' not ', text)
    text = re.sub(r"im", 'i am ', text)
    text = re.sub(r"i'm", 'i am ', text)
    text = re.sub(r"\'re", ' are ', text)
    text = re.sub(r"\'d", ' would ', text)
    text = re.sub(r"\'ll", ' will ', text)
                  
    # Operations and special words  
    text = re.sub('#',' ', text)         
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub('foof', 'food', text)
    text = re.sub('msg', 'message', text)
    text = re.sub(' u ', 'you', text)
    
    # Ponctuation Removal
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    
    text = text.split()
    stop_words = stopwords.words("english")
    text = [tok for tok in text if tok not in stop_words]
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(w) for w in text]
    return ' '.join(text)

In [6]:
# Cleaning Text
df['Review'] = df['Review'].map(lambda x: clean_text(x))

In [7]:

# vocabulary_size = 7000
tk = Tokenizer(num_words=None,char_level=True, oov_token='UNK')
tk.fit_on_texts(df['Review'])
# word_index = tk.word_index
# sequences = tk.texts_to_sequences(df['Review'])


In [8]:
# construct a new vocabulary
alphabet = "abcdefghijklmnopqrstuvwxyz0123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1

# Use char_dict to replace the tk.word_index
tk.word_index = char_dict.copy()
# Add 'UNK' to the vocabulary
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1

In [9]:
# Convert string to index
train_sequences = tk.texts_to_sequences(df['Review'])
# test_texts = tk.texts_to_sequences(test_texts)

# Padding
train_data = pad_sequences(train_sequences, maxlen=1014, padding='post')
# test_data = pad_sequences(test_texts, maxlen=1014, padding='post')

# Convert to numpy array
train_data = np.array(train_data, dtype='float32')
# test_data = np.array(test_data, dtype='float32')
X=train_data

In [10]:
vocab_size=len(tk.word_index)

In [11]:
embedding_weights=[]
embedding_weights.append(np.zeros(vocab_size))

for char, i in tk.word_index.items():
  onehot=np.zeros(vocab_size)
  onehot[i-1]=1
  embedding_weights.append(onehot)
embedding_weights=np.array(embedding_weights)

In [12]:
print(embedding_weights.shape)

(70, 69)


In [13]:
# parameter 
input_size = 1014
# vocab_size = 69
embedding_size = 69
conv_layers = [[256, 7, 3], 
               [256, 7, 3], 
               [256, 3, -1], 
               [256, 3, -1], 
               [256, 3, -1], 
               [256, 3, 3]]

fully_connected_layers = [1024, 1024]
num_of_classes = 4
dropout_p = 0.5
optimizer = 'adam'
loss = 'binary_crossentropy'

In [14]:
# Embedding layer Initialization

embedding_layer = Embedding(vocab_size+1, 
                            embedding_size,
                            input_length=input_size,
                            weights=[embedding_weights])
                            

In [15]:
y = np.array(df[[ 'Components', 'Delivery and Customer Support',
       'Design and Aesthetics', 'Dimensions', 'Features', 'Functionality',
       'Installation', 'Material', 'Price', 'Quality', 'Usability','Polarity']])


In [19]:
def get_model(n_inputs, n_outputs):
  #batch_size = 256 # It is the sample size of inputs to be processed at each training stage. 
  hidden_units = 128
  dropout = 0.45
  inputs = Input(shape=(n_inputs), name='input', dtype='int64')
  # Nossa  MLP com ReLU e Dropout 
  model = Sequential()
 
  model.add(Embedding(vocab_size+1,embedding_size,input_length=n_inputs,weights=[embedding_weights])) #(embedding_layer(inputs))
 
  model.add(Dropout(0.25))
  model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Bidirectional(LSTM(100, return_sequences=True)))
  model.add(Bidirectional(LSTM(100)))
  model.add(Dropout(0.25))
  model.add(Dense(64,activation='relu'))
  model.add(Dense(n_outputs, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  model.summary()
  return model

In [ ]:
'''
def get_model(n_inputs, n_outputs):
  #batch_size = 256 # It is the sample size of inputs to be processed at each training stage. 
  hidden_units = 128
  dropout = 0.45
  inputs = Input(shape=(n_inputs), name='input', dtype='int64')
  # Nossa  MLP com ReLU e Dropout 
  model = Sequential()
 
  model.add(Embedding(vocab_size+1,embedding_size,input_length=n_inputs,weights=[embedding_weights])) #(embedding_layer(inputs))
 
  model.add(Dropout(0.45))
  
  model.add(Bidirectional(LSTM(100, return_sequences=True)))
  model.add(Bidirectional(LSTM(100)))
  model.add(Dropout(0.45))
  model.add(Dense(64,activation='relu'))
  model.add(Dense(n_outputs, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  model.summary()
  return model
  '''

In [22]:
def evaluate_model(X,y):
  results_test = []
  results_train =[]
  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3,min_delta = 0.01)
  n_inputs, n_outputs = X.shape[1], y.shape[1]
  cv = RepeatedKFold(n_splits=3, n_repeats=2, random_state=1)
  for train_ix,test_ix in cv.split(X):
    X_train, X_test = X[train_ix], X[test_ix]
    y_train,y_test = y[train_ix],y[test_ix]
    
    model = get_model(n_inputs, n_outputs)
    
    model.fit(X_train,y_train,epochs = 1,callbacks = callback)
    yhat_train = model.predict(X_train)
    yhat_test = model.predict(X_test)

    train_log_loss = log_loss(y_train, yhat_train)
    test_log_loss = log_loss(y_test,yhat_test)
    
    results_train.append(train_log_loss)
    results_test.append(test_log_loss)

  return results_train,results_test,model

In [ ]:
results_train,results_test,model = evaluate_model(X, y)
print(results_train)
print(results_test)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1014, 69)          4830      
                                                                 
 dropout_2 (Dropout)         (None, 1014, 69)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 1014, 32)          6656      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 507, 32)          0         
 1D)                                                             
                                                                 
 bidirectional_4 (Bidirectio  (None, 507, 200)         106400    
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 200)             

In [ ]:
test_data = pd.read_csv('/content/test.csv')


In [ ]:
pre_processed_test_review =  test_data['Review'].map(lambda x: clean_text(x))

In [ ]:
test_data['preprocessed_test_review'] = pre_processed_test_review

In [ ]:
tk.fit_on_texts(test_data['Review'])

In [ ]:
# Convert string to index
test_sequences = tk.texts_to_sequences(test_data['Review'])
# test_texts = tk.texts_to_sequences(test_texts)

# Padding
test_data = pad_sequences(test_sequences, maxlen=1014, padding='post')
# test_data = pad_sequences(test_texts, maxlen=1014, padding='post')

# Convert to numpy array
test_data = np.array(test_data, dtype='float32')
# test_data = np.array(test_data, dtype='float32')
X_val=test_data

In [ ]:
prediction_on_test_data = model.predict(X_val)

In [ ]:
df = pd.DataFrame(prediction_on_test_data, columns = ['Components', 'Delivery and Customer Support',
       'Design and Aesthetics', 'Dimensions', 'Features', 'Functionality',
       'Installation', 'Material', 'Price', 'Quality', 'Usability',
       'Polarity'])


In [ ]:
df.to_csv('submission_char_cnn.csv', index=False)